In [1]:
# Import modules
import numpy as np
import matplotlib.pyplot as plt

In [50]:
# Create a random 2D state of spins
def initialise_state(N):
    return np.random.choice([-1,1], size=(N,N))

In [67]:
# Calculate energy change when spin(i,j) is flipped, assuming periodic boundary conditions
def delta_E(state, pos_i, pos_j, J, H, N):
    return 2*state[pos_i,pos_j]*(J*(state[pos_i,(pos_j+1)%N] + state[(pos_i-1)%N,pos_j] + state[(pos_i+1)%N,pos_j] + state[pos_i,(pos_j-1)%N])+H)